# Post-processing example with Bunsen burner (Production case in PeleLMeX)

In [ ]:
## Manage the imports
# Graphics
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colors
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Calculation
import numpy as np
from scipy import interpolate
from scipy import stats
from scipy.stats import binned_statistic_dd

# General things
import sys
import pathlib
import copy
import math
import time
import os
import glob

# Python code import
%run "../lib/1-Visualization/SliceReader.py"
%run "../lib/3-Statistics/MultiStatReader.py"

# Load Pele data and plot some relevant fields

In [ ]:
## Directory with the slices (change with whatever you have)
Pele_slice_dir = ("../../custom_PeleAnalysis/Results/slice_2d/tuto_bunsen/")
# Find which plotfiles have been sliced ( sorted from early to latest)
list_plt = sorted(glob.glob(Pele_slice_dir+"plt?????"))
print("{} plotfiles sliced found".format(len(list_plt)))
# take the last one by default and get its basename
select_plt = list_plt[0]
select_plt = list_plt[-1]
base_plt = os.path.basename(select_plt)
print("Last plotfile {} will be plot".format(base_plt))

## slice parameters normal to x direction
axis = "x"
dist = 0.50
level = 2
x_dir = select_plt + "/{}_{:.2f}_{}/".format(axis, dist, level)
# Read slices with the SliceReader classe
Pele_slice = SliceReader(x_dir, file_type="binary", load_list=["temp", "HeatRelease", "FI_CH4_O2"])
Pele_slice.writeInfo()   # write the summary of the slice
# Retrieve the numpy arrays
dict_data_x = Pele_slice.data

## slice parameters normal to z direction
axis = "z"
dist = 0.10
level = 2
z_dir = select_plt + "/{}_{:.2f}_{}/".format(axis, dist, level)
# Read slices with the SliceReader classe
Pele_slice = SliceReader(z_dir, file_type="binary", load_list=["temp", "HeatRelease", "FI_CH4_O2"])
# Retrieve the numpy arrays
dict_data_z = Pele_slice.data

## Make a figure out of that with a custom function in the python file: overbar_plot()
## figsize must be adapted manually, norms can be adjusted too
## extent can be filled with real or normalized dimension
fig, axs = plt.subplots(3, 2, figsize=(16, 24), dpi=400)

## x-normal contours
# Collect and process a bit flame index to only print it where HRR is "high"
HRR = copy.deepcopy(dict_data_x["HeatRelease"])
FI = copy.deepcopy(dict_data_x["FI_CH4_O2"])
FI[HRR<1e7] = np.nan
HRR[HRR<1e7] = 1
# Do the plot
overbar_plot(fig, axs[0,0], dict_data_x["temp"], cmap="RdBu_r", norm=colors.Normalize(), extent=None)
overbar_plot(fig, axs[1,0], HRR, cmap="hot", norm=colors.LogNorm(1e7, 1e10), extent=None)
overbar_plot(fig, axs[2,0], FI, cmap="PiYG", norm=colors.Normalize(), extent=None)

# Figure formatting
axs[0,0].set_title("x* = 0.5\nT [K]", y=1.16)
axs[1,0].set_title(r"Heat release rate [W/m$^3$]", y=1.16)
axs[2,0].set_title("CH4 flame index", y=1.16)
#axs[0].set_xlabel("y")
#axs[1].set_xlabel("y")
axs[2,0].set_xlabel("y")
axs[0,0].set_ylabel("z")
axs[1,0].set_ylabel("z")
axs[2,0].set_ylabel("z")

## z-normal contours
# Collect and process a bit flame index to only print it where HRR is "high"
HRR = copy.deepcopy(dict_data_z["HeatRelease"])
FI = copy.deepcopy(dict_data_z["FI_CH4_O2"])
FI[HRR<1e7] = np.nan
HRR[HRR<1e7] = 1
# Do the plot
overbar_plot(fig, axs[0,1], dict_data_z["temp"], cmap="RdBu_r", norm=colors.Normalize(), extent=None)
overbar_plot(fig, axs[1,1], HRR, cmap="hot", norm=colors.LogNorm(1e7, 1e10), extent=None)
overbar_plot(fig, axs[2,1], FI, cmap="PiYG", norm=colors.Normalize(), extent=None)

# Figure formatting
axs[0,1].set_title("z* = 0.1\nT [K]", y=1.16)
axs[1,1].set_title(r"Heat release rate [W/m$^3$]", y=1.16)
axs[2,1].set_title("CH4 flame index", y=1.16)
#axs[0].set_xlabel("y")
#axs[1].set_xlabel("y")
axs[2,1].set_xlabel("x")
axs[0,1].set_ylabel("y")
axs[1,1].set_ylabel("y")
axs[2,1].set_ylabel("y")

fig.tight_layout()
plt.show()

# Now load all available plotfiles and save images (to prepare for an animation :D)
An animation can be created with a quick bash script in lib/2-Movie, but other method exist to do that

In [ ]:
## Directory with the slices (change with whatever you have)
Pele_slice_dir = ("../../custom_PeleAnalysis/Results/slice_2d/tuto_bunsen/")
# Find which plotfiles have been sliced ( sorted from early to latest)
list_plt = sorted(glob.glob(Pele_slice_dir+"plt?????"))
print("{} plotfiles sliced found".format(len(list_plt)))
print("all plotfiles will be plot")

# output directory for the images
output_dir = ("../Results/images/tuto_bunsen/contours/")
pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

for select_plt in list_plt:
    # Get base plt name for name of the image to plot
    base_plt = os.path.basename(select_plt)
    
    ## slice parameters normal to x direction
    axis = "x"
    dist = 0.50
    level = 2
    x_dir = select_plt + "/{}_{:.2f}_{}/".format(axis, dist, level)
    # Read slices with the SliceReader classe
    Pele_slice = SliceReader(x_dir, file_type="binary", load_list=["temp", "HeatRelease", "FI_CH4_O2"])
    #Pele_slice.writeInfo()   # write the summary of the slice
    sim_time = Pele_slice.time
    # Retrieve the numpy arrays
    dict_data_x = Pele_slice.data

    ## slice parameters normal to z direction
    axis = "z"
    dist = 0.10
    level = 2
    z_dir = select_plt + "/{}_{:.2f}_{}/".format(axis, dist, level)
    # Read slices with the SliceReader classe
    Pele_slice = SliceReader(z_dir, file_type="binary", load_list=["temp", "HeatRelease", "FI_CH4_O2"])
    # Retrieve the numpy arrays
    dict_data_z = Pele_slice.data

    ## Make a figure out of that with a custom function in the python file: overbar_plot()
    ## figsize must be adapted manually, norms can be adjusted too
    ## extent can be filled with real or normalized dimension
    fig, axs = plt.subplots(3, 2, figsize=(16, 24), dpi=100)
    fig.suptitle("t = {:.4f} s".format(sim_time))
    
    ## x-normal contours
    # Collect and process a bit flame index to only print it where HRR is "high"
    HRR = copy.deepcopy(dict_data_x["HeatRelease"])
    FI = copy.deepcopy(dict_data_x["FI_CH4_O2"])
    FI[HRR<1e7] = np.nan
    HRR[HRR<1e7] = 1
    # Do the plot
    overbar_plot(fig, axs[0,0], dict_data_x["temp"], cmap="RdBu_r", norm=colors.Normalize(), extent=None)
    overbar_plot(fig, axs[1,0], HRR, cmap="hot", norm=colors.LogNorm(1e7, 1e10), extent=None)
    overbar_plot(fig, axs[2,0], FI, cmap="PiYG", norm=colors.Normalize(), extent=None)

    # Figure formatting
    axs[0,0].set_title("x* = 0.5\nT [K]", y=1.16)
    axs[1,0].set_title(r"Heat release rate [W/m$^3$]", y=1.16)
    axs[2,0].set_title("CH4 flame index", y=1.16)
    #axs[0].set_xlabel("y")
    #axs[1].set_xlabel("y")
    axs[2,0].set_xlabel("y")
    axs[0,0].set_ylabel("z")
    axs[1,0].set_ylabel("z")
    axs[2,0].set_ylabel("z")

    ## z-normal contours
    # Collect and process a bit flame index to only print it where HRR is "high"
    HRR = copy.deepcopy(dict_data_z["HeatRelease"])
    FI = copy.deepcopy(dict_data_z["FI_CH4_O2"])
    FI[HRR<1e7] = np.nan
    HRR[HRR<1e7] = 1
    # Do the plot
    overbar_plot(fig, axs[0,1], dict_data_z["temp"], cmap="RdBu_r", norm=colors.Normalize(), extent=None)
    overbar_plot(fig, axs[1,1], HRR, cmap="hot", norm=colors.LogNorm(1e7, 1e10), extent=None)
    overbar_plot(fig, axs[2,1], FI, cmap="PiYG", norm=colors.Normalize(), extent=None)

    # Figure formatting
    axs[0,1].set_title("z* = 0.1\nT [K]", y=1.16)
    axs[1,1].set_title(r"Heat release rate [W/m$^3$]", y=1.16)
    axs[2,1].set_title("CH4 flame index", y=1.16)
    #axs[0].set_xlabel("y")
    #axs[1].set_xlabel("y")
    axs[2,1].set_xlabel("x")
    axs[0,1].set_ylabel("y")
    axs[1,1].set_ylabel("y")
    axs[2,1].set_ylabel("y")
    
    # Save the image and close it
    fig.tight_layout()
    plt.savefig(output_dir + "multiplot_{}.png".format(base_plt), dpi=200)
    #plt.show()  # don't show or there will be too many images on the interface
    plt.close()

# Load amrex_kitchen slices and do the same (TODO)

# Read statistics written by Pele

In [ ]:
## Directory with the slices (change with whatever you have)
Pele_stat_dir = ("../../custom_PeleAnalysis/Results/stats/tuto_bunsen/")
## Variables to load
main_var = "HeatRelease"
var1 = "mixture_fraction"
var2 = "temp"
var3 = "HeatRelease"
type_stats = "X1-{}_X2-{}_X3-{}/".format(var1, var2, var3)

## Find all plotfiles available with stats
path_stat_plt = Pele_stat_dir + type_stats
list_plt = sorted(glob.glob(path_stat_plt + "plt?????"))
N_plt = len(list_plt)
print("{} plotfiles sliced found".format(len(list_plt)))
# take the last one by default and get its basename
select_plt = list_plt[-1]
base_plt = os.path.basename(select_plt)
print("Last plotfile {} will be loaded".format(base_plt))

## load the MultiStat object
verbose = False    # print all details of operation if True
MS = MultiStat(select_plt, main_var, verbose=verbose)

# Reduce the heat release rate dimension if we want only 1D statistics conditioned on temperature
MS_2d = MS.reduceToNDim(["mixture_fraction", "temp"])

## Use some function to plot things
# JPDF temp/HRR with marginal plots
fig = MS_2d.marginalPlot()
plt.close()

In [ ]:
## Directory with the slices (change with whatever you have)
Pele_stat_dir = ("../../custom_PeleAnalysis/Results/stats/tuto_bunsen/")
## Variables to load
main_var = "HeatRelease"
var1 = "mixture_fraction"
var2 = "temp"
var3 = "HeatRelease"
type_stats = "X1-{}_X2-{}_X3-{}/".format(var1, var2, var3)

## Find all plotfiles available with stats
path_stat_plt = Pele_stat_dir + type_stats
list_plt = sorted(glob.glob(path_stat_plt + "plt?????"))
N_plt = len(list_plt)
print("{} plotfiles sliced found".format(len(list_plt)))
# take the last one by default and get its basename
select_plt = list_plt[-1]
base_plt = os.path.basename(select_plt)
print("Last plotfile {} will be loaded".format(base_plt))

## load the MultiStat object
verbose = False    # print all details of operation if True
MS = MultiStat(select_plt, main_var, verbose=verbose)

# This function can be used to further filter based on conditioning quantity
# here a range of mixture fraction
MS_filter = MS.filterSingleDimension("mixture_fraction", [0.05, 0.06])

# Then reduce dimension to temp/HRR
MS_2d = MS_filter.reduceToNDim(["temp", "HeatRelease"])

# JPDF in T/HRR space (the 0D solution could be superimposed almost perfectly)
fig, ax = plt.subplots(1, 1, figsize=(8, 8), dpi=100) 
MS_2d.plot2d(fig, ax, type_stat="pdf")

# Now get the mean HRR conditioned on T and superimpose it on top
MS_1d = MS_filter.reduceToNDim(["temp"])
X = MS_1d.list_bin[0].value_bin
HRR_mean = np.log10(MS_1d.data_mean)
HRR_max = np.log10(MS_1d.data_max)
ax.plot(X, HRR_mean, color="black", linewidth=3, label="mean")
ax.plot(X, HRR_max, color="red", linewidth=3, linestyle="--", label="max")

ax.set_title("JPDF")
ax.set_xlabel("T [K]")
ax.set_ylabel(r"Heat release rate [W/m$^3$]")
ax.set_ylim(np.nanmin(HRR_max), 1.02*np.nanmax(HRR_max))
ax.legend()

fig.tight_layout()
plt.show()
plt.close()